run setup scripts to configure catalogs and schemas, paths, datasets

In [0]:
%sql
use catalog `get_started`;
use schema `labuser`;

SELECT * FROM parquet.`/Volumes/get_started/labuser/myfiles/sales-historical` LIMIT 10

In [0]:
%sql
-- drop table if exists current_employees
DROP TABLE IF EXISTS historical_sales_bronze;
    
-- create a delta table
CREATE TABLE historical_sales_bronze USING DELTA
AS
SELECT * FROM parquet.`/Volumes/get_started/labuser/myfiles/sales-historical`;

-- Display available tables in schema
SHOW TABLES

In [0]:
%sql
DESCRIBE historical_sales_bronze;

In [0]:
%sql
-- drop table if exists current_employees
DROP TABLE IF EXISTS current_employees_bronze;
    
-- create a delta table
CREATE TABLE current_employees_bronze USING DELTA
AS
SELECT *
FROM read_files(
    '/Volumes/get_started/labuser/myfiles/employees.csv',
    format => 'csv',
    sep => ",",
    header => true,
    inferSchema => true,
    mode => "FAILFAST"
);

-- Display available tables in schema
SHOW TABLES

In [0]:
%sql
DESCRIBE EXTENDED current_employees_bronze;

In [0]:
%sql
SELECT * FROM current_employees_bronze LIMIT 5;

Load Incrementally

In [0]:
%sql
-- drop table if exists current_employees
DROP TABLE IF EXISTS historical_sales_copyinto_bronze;
    
-- create a delta table
CREATE TABLE historical_sales_copyinto_bronze USING DELTA;

In [0]:
%sql
COPY INTO historical_sales_copyinto_bronze
FROM '/Volumes/get_started/labuser/myfiles/sales-historical'
FILEFORMAT = parquet
-- FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');

Creating external tables

In [0]:
%sql
-- create a delta table externally
CREATE TABLE current_employees_bronze USING CSV OPTIONS (
  header = "true",
  delimeter = ",",
)
LOCATION 'path to the pre-configured external location';


Built-in Functions
Databricks has vast number of builtin Functions

In [0]:
%sql
-- CREATE TABLE users_bronze;

-- COPY INTO users_bronze FROM
-- (SELECT *,
-- cast(cast(user_first_touch_timestamp/1e6 AS TIMESTAMP) AS DATE) first_touch_date,
-- current_timestamp() updated,
-- input_file_name() source_file
-- FROM '${da.paths.datasets}/ecommerce/raw/users-historical/')
-- FILEFORMAT = PARQUET
-- COPY_OPTIONS ('mergeSchema' = 'true');


-- SELECT * FROM users_bronze LIMIT 10;

In [0]:
%python
# DA.cleanup()